# Proyecto N°1 con Pandas - Análisis de datos exploratorios en un set de datos de Películas

## Importación de datos e inspección preliminar

In [1]:
import pandas as pd # Importa la libreria de pandas, atplotlib y numpy
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_columns = 30 #aumenta las columnas a mostrar a 30
pd.options.display.float_format = '{:.2f}'.format #selecciono que se muestren 2 decimales

In [2]:
df=pd.read_csv('movies_complete.csv', parse_dates=['release_date'])
#asigno a df el csv con la base de datos de películas y transformo a fecha la columna 'release_date'

In [ ]:
df.head()
#muestro las 5 primeras filas del DataFrame

In [ ]:
df.info()
#revisamos la información básica del DataFrame

#### Descripción de cada columna
* **id:** The ID of the movie (clear/unique identifier).
* **title:** The Official Title of the movie.
* **tagline:** The tagline of the movie.
* **release_date:** Theatrical Release Date of the movie.
* **genres:** Genres associated with the movie.
* **belongs_to_collection:** Gives information on the movie series/franchise the particular film belongs to.
* **original_language:** The language in which the movie was originally shot in.
* **budget_musd:** The budget of the movie in million dollars.
* **revenue_musd:** The total revenue of the movie in million dollars.
* **production_companies:** Production companies involved with the making of the movie.
* **production_countries:** Countries where the movie was shot/produced in.
* **vote_count:** The number of votes by users, as counted by TMDB.
* **vote_average:** The average rating of the movie.
* **popularity:** The Popularity Score assigned by TMDB.
* **runtime:** The runtime of the movie in minutes.
* **overview:** A brief blurb of the movie.
* **spoken_languages:** Spoken languages in the film.
* **poster_path:** The URL of the poster image.
* **cast:** (Main) Actors appearing in the movie.
* **cast_size:** number of Actors appearing in the movie.
* **director:** Director of the movie.
* **crew_size:** Size of the film crew (incl. director, excl. actors).

In [ ]:
df.describe()
# Realiza un análisis descriptivo general de los atos numéricos del DataFrame

In [ ]:
df.hist(figsize=(20,12),bins=100)
plt.show
# creamos un histograma de los datos, así entendemos la distribución del los datos

In [20]:
df.spoken_languages.value_counts(dropna=False).head(10)
# Con esto buscamos los presupuestos más comunes en nuestro DataFrame y mostramos los 10 primeros 
#budget - N° valores

English             22189
NaN                  3597
Français             1831
日本語                  1278
Italiano             1197
Español               891
Pусский               797
Deutsch               751
English|Français      677
English|Español       569
Name: spoken_languages, dtype: int64

In [ ]:
df.revenue_musd.value_counts(dropna=False).head(10)

In [ ]:
df.vote_count.value_counts(dropna=False).head(10)

In [ ]:
df.vote_average.value_counts(dropna=False).head(10)

In [ ]:
df.describe(include=object)
# Veo las columnas no numéricas y puedo ver valores no nulos, valores únicos, etc.

In [ ]:
#Con lo anterior, vimos que el title más repetido es 'Cinderella', vamos a comprobarlo
df[df.title=='Cinderella']

## La mejor y la peor película

In [10]:
from IPython.display import HTML
#Con esto permitimos mostrar contenido HTML en el notebook

In [ ]:
df_best = df[["poster_path", "title", "budget_musd", "revenue_musd",
              "vote_count", "vote_average", "popularity"]].copy()
df_best
# Creamos un nuevo DataFrame copiando ciertas columnas desde el DataFrame principal 

In [36]:
df_best['profit_musd']=df.revenue_musd.sub(df.budget_musd)
df_best['return']=df.revenue_musd.div(df.budget_musd)
# Creo 2 nuevas columnas en el DataFrame 'df_best', en la primera resto y en la otra divido.

In [ ]:
df_best

In [38]:
df_best.columns = ["", "Title", "Budget", "Revenue", "Votes", 
                   "Average Rating", "Popularity", "Profit", "ROI"]
# Renombramos las columnas con fines estéticos

In [39]:
df_best.set_index('Title', inplace=True )
# Elegimos el 'Title' como index del DataFrame

In [ ]:
df_best

**Solo con fines de demostrar las imágenes**

In [ ]:
subset = df_best.iloc[:5, :2]
subset
# separamos solamente las columnas  Title, la que contiene las url de las img y el budget.

In [ ]:
HTML(subset.to_html(escape=False))
# Muestra en HTML las imágenes

---

In [ ]:
df_best.loc[df_best.Budget >= 5].sort_values(by = "ROI", ascending = False)
# Selecciono las películas con un budget mayor a 5 y ordeno por ROI

In [51]:
# Ya que existían mucho null, rellenamos las columnas de Budget y Votes con valores 0
df_best.Budget.fillna(0, inplace = True)
df_best.Votes.fillna(0, inplace = True)
#inplace= rue se usa para reemplazar en el DataFrame original, en caso contrario debemos asignar a nueva variable

In [ ]:
df_best.info() # revisamos lo anterior

In [55]:
# se crea una función para determinar la mejor y peor. n: número de peliculas; by: criterio;
# ascending: peor o mejor; min_bud: presupuesto minimo; min_votes: votos mínimos.
def best_worst(n, by, ascending = False, min_bud = 0, min_votes = 0):
    
    
    df2 = df_best.loc[(df_best.Budget >= min_bud) & (df_best.Votes >= min_votes), 
                      ["", by]].sort_values(by = by, ascending = ascending).head(n).copy()
    
    return HTML(df2.to_html(escape=False))

**Top 5 películas con mayor Revenue**

In [ ]:
best_worst(n = 5, by = "Revenue")

**Top 5 películas con mayor Budget**

In [ ]:
best_worst(5, "Budget")

**Top 5 con mayores Profit**

In [ ]:
best_worst(5, "Profit")

**Top 5 con menor Rating** (Min 20 votos y Min 20 musd)

In [ ]:
best_worst(5, "Average Rating", ascending = True, min_votes = 20, min_bud = 20)

**Top 5 más populares**

In [ ]:
best_worst(5, "Popularity")

## Encuentra tu siguiente película

**Búsqueda 1: Películas de ciencia ficción y acción que sean protagonizadas por Bruce Willis**

In [4]:
df.genres[0]

'Animation|Comedy|Family'

In [ ]:
mask_genres = df.genres.str.contains("Action") & df.genres.str.contains("Science Fiction")
mask_genres
# Crea un DF con los géneros que son de acción y de ciencia ficción.

In [6]:
df.cast[0]

'Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wallace Shawn|John Ratzenberger|Annie Potts|John Morris|Erik von Detten|Laurie Metcalf|R. Lee Ermey|Sarah Freeman|Penn Jillette'

In [ ]:
mask_actor = df.cast.str.contains("Bruce Willis")
mask_actor
# Crea un DF con las películas que tienen a Bruce Willis cómo actor.

In [ ]:
df.loc[mask_actor & mask_genres, ["title", "vote_average"]].sort_values(by = "vote_average", 
                                                                        ascending = False)
# Combina los DF de actor y genre, además agrega los títulos y el voto promedio. Ordena en forma descendente

In [ ]:
bruce=df.loc[mask_actor & mask_genres, ["title","poster_path", "vote_average"]].sort_values(by = "vote_average", 
                                                                        ascending = False)
HTML(bruce.to_html(escape=False))

**Búsqueda 2: Películas de comedia y animación que se hablen en alemán, ordenadas por duración**

In [ ]:
df_genre= df.genres.str.contains("Comedy")&df.genres.str.contains("Animation")
df_genre

In [ ]:
df_language=df.spoken_languages.str.contains("Deutsch")
df_language

In [ ]:
movie=df.loc[df_genre&df_language,['title','poster_path','runtime']].sort_values(by='runtime', ascending=False)
HTML(movie.to_html(escape=False))

**Búsqueda 3: Películas de acción o Thriller con idioma original en Ingles, con un rating promedio mínimo de 6**

In [ ]:
df_genre= df.genres.str.contains("Action")|df.genres.str.contains("Thriller")
df_genre

In [ ]:
df_language=df.original_language=="en"
df_language

In [27]:
df_vote=df.vote_average>6

In [ ]:
movie=df.loc[df_genre&df_language&df_vote,['title','poster_path','runtime']].sort_values(by='runtime', ascending=False)
HTML(movie.to_html(escape=False))

## Palabras más comunes en los títulos y en el tagline

In [36]:
from wordcloud import WordCloud

In [ ]:
df

In [40]:
df.tagline[1]

'Roll the dice and unleash the excitement!'

In [41]:
title = df.title.dropna()
overview = df.overview.dropna()
tagline = df.tagline.dropna()
# Con esto evito que existan valores nulos en la nube de etiquetas.

In [ ]:
' '.join(title)
#Esto une todos los titulos separados por un ' '

In [43]:
title_corpus = ' '.join(title)
overview_corpus = ' '.join(overview)
tagline_corpus = ' '.join(tagline)
# Junta todo en un simple string

In [ ]:
tagline_corpus

In [45]:
title_wordcloud = WordCloud(background_color='white', height=2000, width=4000, max_words= 200).generate(title_corpus)
title_wordcloud
#Esto genera la nube de etiquetas, pero no lo dibuja

In [ ]:
plt.figure(figsize=(16,8))
plt.imshow(title_wordcloud, interpolation= "bilinear")
plt.axis('off')
plt.show()
#Con esto dibujo la nube de etiquetas.

In [ ]:
tagline_wordcloud = WordCloud(background_color='white', height=2000, width=4000).generate(tagline_corpus)
plt.figure(figsize=(16,8))
plt.imshow(tagline_wordcloud, interpolation= "bilinear")
plt.axis('off')
plt.show()
#Genera la nube de etiquetas del tagline

In [ ]:
overview_wordcloud = WordCloud(background_color='white', height=2000, width=4000).generate(overview_corpus)
plt.figure(figsize=(16,8))
plt.imshow(overview_wordcloud, interpolation= "bilinear")
plt.axis('off')
plt.show()
#Genera la nube de etiquetas del overview

----

## ¿Las franquicias son más exitosas?

In [3]:
df["Franchise"] = df.belongs_to_collection.notna()
# Creo la columna de franquicia, a partir de la columna de pertenece a una colección de los valores no nulos.

In [ ]:
df.Franchise.value_counts()
# Cuento las películas que son una franquicia

**Métricas**

In [ ]:
df.groupby("Franchise").revenue_musd.mean()
#Agrupo de la columna de franquicia los ingresos promedio

In [ ]:
df["ROI"] = df.revenue_musd.div(df.budget_musd)#Creo una columna con el ROI.
df.groupby("Franchise").ROI.median() #Vemos la mediana del ROI

In [ ]:
df.groupby("Franchise").budget_musd.mean()
#Determinamos el presupuesto promedio

In [ ]:
df.groupby("Franchise").popularity.mean()
#Popularidad media de las películas que son franquicias

In [ ]:
df.groupby("Franchise").agg({"budget_musd": "mean", "revenue_musd": "mean", "vote_average": "mean",
                            "popularity": "mean", "ROI":"median", "vote_count":"mean"})
#Creo una agrupación de varios elementos, con el promedio en cada uno de ellos o la mediana.

----

## Los directores más exitosos

In [ ]:
df.director.value_counts().head(20)
#Cuento las películas que tiene cada director.

In [ ]:
plt.figure(figsize = (12, 4))
df.director.value_counts().head(20).plot(kind='bar', fontsize = 12)
plt.title("Directores más activos",fontsize = 20)
plt.ylabel("N° de películas", fontsize = 15)
plt.show()
#Dibujamos un gráfico de barras con los directores más activos

In [ ]:
df.groupby("director").revenue_musd.sum().nlargest(20)
#Agrupamos los directores por los que generaron mayores ingresos (20)


In [ ]:
plt.figure(figsize = (12, 4))
df.groupby("director").revenue_musd.sum().nlargest(20).plot(kind='bar', fontsize = 12)
plt.title("Ingresos totales",fontsize = 20)
plt.ylabel("Ingresos (en MUSD)", fontsize = 15)
plt.show()
#Graficamos los 20 directores con mayores ingresos

In [27]:
directors = df.groupby("director").agg({"title": "count", "vote_average" :"mean", "vote_count": "sum"})
#Agrupamos los directores y agregamos de acuerdo a los criterios.

In [ ]:
directors[(directors.vote_count >= 10000) & (directors.title >= 10)].nlargest(20, "vote_average")
#Genero un subconjunto que cumpla las dos condiciones, luego muestro los 20 mayores vote_average

In [29]:
df.genres = df.genres.astype(str)
#Transformo la columna a String

In [ ]:
df.loc[df.genres.str.contains("Horror")].groupby("director").revenue_musd.sum().nlargest(20)
#Busco en los generos, lo que contenga "Horror" y agrupo los directores. Sumo el revenue y selecciono lo 20 mayores